In [1]:
from mesa import Agent, Model 
from mesa.space import MultiGrid
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

import numpy as np
import pandas as pd
import random

import time
import datetime

In [16]:
def gridCreate(myModel):
    myGrid = np.zeros((myModel.grid.width, myModel.grid.height))
    for cell in myModel.grid.coord_iter():
        contentCell, x, y = cell
        for thisCont in contentCell:
            if isinstance(thisCont, Robot):
                myGrid[x][y] = 2 
            else:
                myGrid[x][y] = thisCont.status
    return myGrid
    
class Robot(Agent):
    
    def __init__(self, myid, myModel):
        super().__init__(myid, myModel)
        self.nextone = None
        self.hist = 0
        
    def step(self):
        closeCells = self.model.grid.get_neighbors(
            self.pos,
            moore=True,
            include_center=True)
        
        for near in closeCells:
            if isinstance(near, Space) and near.pos == self.pos:
                near.tmpstts = near.status
                if near.tmpstts == near.dark: 
                    near.tmpstts = near.clear
                    self.nextone = self.pos
                else:
                    around = self.model.grid.get_neighborhood(
                        self.pos,
                        moore=True,
                        include_center=False)
                    nextone = self.random.choice(around)
                    self.nextone = nextone
                break
        
    def advance(self):
        
       
        closeCells = self.model.grid.get_neighbors(
            self.pos,
            moore=False,
            include_center=True)
        
        for near in closeCells:
            if isinstance(near, Space) and near.pos == self.pos: 
                near.status = near.tmpstts
                break
        
        if self.pos != self.nextone:
            self.hist = self.hist + 1
            
       
        self.model.grid.move_agent(self, self.nextone)
    
class Space(Agent):
    
    dark = 1
    clear = 0
    
    def __init__(self, pos, myModel, status=clear):
        super().__init__(pos, myModel)
        self.x, self.y = pos
        self.status = status
        self.tmpstts = None

class Build(Model):
    
    def __init__(self, m, n, robotCount, spawnDark):
        self.robotCount = robotCount
        self.spawnDark = spawnDark
        self.end_count = 1 - spawnDark
        self.grid = MultiGrid(m, n, True)
        self.schedule = SimultaneousActivation(self)
               
      
        darkcells = int((m * n) * spawnDark)
        lista_cells_vacias = list(self.grid.empties)
        for cells in range(darkcells):
            emptyCell = random.choice(lista_cells_vacias)
            space = Space(emptyCell, self)
            space.status = Space.dark
            self.grid.place_agent(space, emptyCell)
            self.schedule.add(space)
            lista_cells_vacias.remove(emptyCell)
        
      
        lista_cells_vacias = list(self.grid.empties)
        for cells in lista_cells_vacias:
            space = Space(cells, self)
            self.grid.place_agent(space, cells)
            self.schedule.add(space)
        
        for i in range(robotCount):
            robot = Robot(i, self)
            self.grid.place_agent(robot, (1,1))
            self.schedule.add(robot)
            
        self.colectordatos = DataCollector(
            model_reporters={'myGrid': gridCreate},
            agent_reporters={'hist': lambda a: getattr(a, 'hist', None)}
        )
    
    def step(self):
        self.colectordatos.collect(self)
        self.schedule.step()    
    
    def finish(self):
        counterPos = 0
        for cell in self.grid.coord_iter():
            contentCell, x, y = cell
            for thisCont in contentCell:
                if isinstance(thisCont, Space) and thisCont.status == thisCont.clear:
                    counterPos = counterPos + 1
        
        self.end_count = counterPos / (self.grid.width * self.grid.height)
        if self.end_count == 1:
            return True
        else:
            return False




t1 = time.time()
myModel = Build(10, 10, 3, 0.5)

while((time.time() - t1) < 0.5 and not myModel.finish()):
    myModel.step()


tf = str(datetime.timedelta(seconds=(time.time() - t1)))

dataset = myModel.colectordatos.get_model_vars_dataframe()



AttributeError: type object 'Robot' has no attribute 'status'

In [3]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(dataset.iloc[0][0], cmap='Greys')
 